In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

import model_helpers as mh
import numpy as np

# np.set_printoptions(suppress=True)

import plotly.express as px
import plotly.figure_factory as ff

### Model of the lung's health

In [2]:
U = mh.variableNode("Unblocked FEV1", 2, 6, 0.1)
C = mh.variableNode("Small airway clearance", 0.9, 1, 0.1)
FEV1 = mh.variableNode("FEV1", 0.2, 6, 0.1)

graph = BayesianNetwork([(U.name, FEV1.name), (C.name, FEV1.name)])

cpt_fev1 = TabularCPD(
    variable=FEV1.name,
    variable_card=len(FEV1.bins) - 1,
    values=mh.calc_pgmpy_cpt(U, C, FEV1),
    evidence=[C.name, U.name],
    evidence_card=[len(C.bins) - 1, len(U.bins) - 1],
)

prior_b = TabularCPD(
    variable=C.name,
    variable_card=len(C.bins) - 1,
    values=C.uniform_prior(C),
    evidence=[],
    evidence_card=[],
)

prior_u = TabularCPD(
    variable=U.name,
    variable_card=len(U.bins) - 1,
    values=U.uniform_prior(U),
    evidence=[],
    evidence_card=[],
)

graph.add_cpds(cpt_fev1, prior_b, prior_u)

graph.check_model()

inference = BeliefPropagation(graph)

calculating cpt of shape 58 x 40 x 1 (C x (A x B)) 


### Interactive inference

In [7]:
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)

app.layout = html.Div(
    [
        html.H4("Interactive inference with FEV1, Unblocked FEV1"),
        dcc.Graph(id="graph"),
        html.P("FEV1:"),
        dcc.Slider(
            id="fev1",
            min=FEV1.bins[0],
            max=FEV1.bins[-2],
            value=0.2,
            marks={0: "0.2", (len(C.bins) - 1): "5.9"},
        ),
    ]
)


@app.callback(Output("graph", "figure"), Input("fev1", "value"))
def display_color(fev1):
    print("set FEV1 to", fev1)

    [_fev1_bin, fev1_idx] = mh.get_bin_for_value(fev1, FEV1.bins)

    res = inference.query(variables=[U.name], evidence={FEV1.name: fev1_idx})
    fig = px.bar(y=res.values, x=U.bins[:-1])
    fig.update_layout(
        xaxis_title="Distribution of unblocked FEV1 (L)",
        yaxis_title="Probability",
        legend_title="Legend Title",
        font=dict(family="Courier New, monospace", size=12, color="#7f7f7f"),
    )
    return fig


app.run_server(debug=True, port=8049, use_reloader=False)

Dash is running on http://127.0.0.1:8049/

 * Serving Flask app '__main__'
 * Debug mode: on
get fev1 0.2
fev1_idx 0
fev1_bin [0.2; 0.30000000000000004[


  0%|          | 0/1 [00:00<?, ?it/s]

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:477: RuntimeWarning:

invalid value encountered in true_divide



get fev1 0.2
fev1_idx 0
fev1_bin [0.2; 0.30000000000000004[


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

get fev1 5.4
fev1_idx 52
fev1_bin [5.400000000000002; 5.500000000000002[


  0%|          | 0/1 [00:00<?, ?it/s]

get fev1 4.6
fev1_idx 44
fev1_bin [4.600000000000001; 4.700000000000002[


  0%|          | 0/1 [00:00<?, ?it/s]

0.1-0.05

In [4]:
FEV1.b


['[2.1000000000000005; 2.200000000000001[', 19]